https://jira.x5.ru/browse/CVMXC-2633

In [1]:
name = 'CVMXC-2633_immunele_pa'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### X-28-26-Pepsi-Imunele-3plu-sof

In [6]:
campaign_id = 'X-28-26-Pepsi-Imunele-3plu-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        79931: "sms_viber_tg",
        79931: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-18',
    'date_communication_to': '2022-01-24',

    'wave_ids': [
        80030 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-18',
    'date_to': '2022-01-24', #дата постпериода
    'control_camp_wave_id': 79931,
    'plu_list': list(map(str, [3959,4159731,2137009,2137039,3398892,3426770,3426771,4054537,3426773,3373862,
                 3270027,3270028,3356409,3004331,3929853,3398891,4150434,3962,3501805,3326481,
                 3342907,3270022,3929854,3662340,3270024,4054541,3961,3971,3675339,2137162,
                 4054540,3662339,2137159,3675338,3426774,4004,12260,3501804,3675337,4054539,
                 3929852,3342908,3426772,2157641,3662341,3229726,4150433,46222])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 3 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [7]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [8]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [9]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [10]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
1_79931_sms_viber_cg,28947,"7,967.98",0.00,0.00,0.00,0.00,"773,324.55","18,397.00","6,941.00",24986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0_79931_sms_viber_cg,115781,"148,834.89",1.36,"157,653.12",1.38,"159,798.00","3,107,168.16","77,511.00","31,494.00",100236,...,"276,241.00",0.00,0.00,0.44,"50,560.00",0.07,0.00,0.38,"44,385.00",0.33


In [11]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### X-28-27-Pepsi-Imunele-6plu-sof

In [12]:
campaign_id = 'X-28-27-Pepsi-Imunele-6plu-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        79923: "sms_viber_tg",
        79923: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-18',
    'date_communication_to': '2022-01-24',

    'wave_ids': [
        80069 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-18',
    'date_to': '2022-01-24', #дата постпериода
    'control_camp_wave_id': 79923,
    'plu_list': list(map(str, [3959,4159731,2137009,2137039,3398892,3426770,3426771,4054537,3426773,3373862,
                 3270027,3270028,3356409,3004331,3929853,3398891,4150434,3962,3501805,3326481,
                 3342907,3270022,3929854,3662340,3270024,4054541,3961,3971,3675339,2137162,
                 4054540,3662339,2137159,3675338,3426774,4004,12260,3501804,3675337,4054539,
                 3929852,3342908,3426772,2157641,3662341,3229726,4150433,46222])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 6 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [13]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [14]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [15]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [16]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_79923_sms_viber_cg,115692,"110,970.84",1.36,"157,514.88",1.38,"159,588.00","3,099,755.85","79,127.00","28,058.00",100350,...,"-61,410.00",-0.00,0.00,0.48,"55,719.00",0.08,0.00,0.52,"60,566.00",0.39
1_79923_sms_viber_cg,28930,"-1,503.62",0.00,0.00,0.00,0.00,"775,862.52","19,694.00","6,737.00",25034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### U-28-immunele-3ps-postavschik

In [18]:
campaign_id = 'U-28-immunele-3ps-postavschik'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        79909: "sms_viber_tg",
        79909: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-18',
    'date_communication_to': '2022-01-24',

    'wave_ids': [
        80057 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-18',
    'date_to': '2022-01-24', #дата постпериода
    'control_camp_wave_id': 79909,
    'plu_list': list(map(str, [3959,4159731,2137009,2137039,3398892,3426770,3426771,4054537,3426773,3373862,
                 3270027,3270028,3356409,3004331,3929853,3398891,4150434,3962,3501805,3326481,
                 3342907,3270022,3929854,3662340,3270024,4054541,3961,3971,3675339,2137162,
                 4054540,3662339,2137159,3675338,3426774,4004,12260,3501804,3675337,4054539,
                 3929852,3342908,3426772,2157641,3662341,3229726,4150433,46222])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 3 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [19]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [20]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [21]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [22]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_79909_sms_viber_cg,94437,"150,141.61",1.36,"128,257.80",1.47,"138,516.00","3,355,227.60","69,027.00","24,334.00",86619,...,"-371,435.00",-0.01,0.00,0.29,"27,529.00",0.04,0.00,0.42,"39,137.00",0.47
1_79909_sms_viber_cg,40603,"20,928.89",0.00,0.00,0.00,0.00,"1,455,599.63","29,475.00","9,594.00",37304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### U-28-immunele-6ps-postavschik

In [24]:
campaign_id = 'U-28-immunele-6ps-postavschik'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        79946: "sms_viber_tg",
        79909: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-18',
    'date_communication_to': '2022-01-24',

    'wave_ids': [
        80065 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-18',
    'date_to': '2022-01-24', #дата постпериода
    'control_camp_wave_id': 79909,
    'plu_list': list(map(str, [3959,4159731,2137009,2137039,3398892,3426770,3426771,4054537,3426773,3373862,
                 3270027,3270028,3356409,3004331,3929853,3398891,4150434,3962,3501805,3326481,
                 3342907,3270022,3929854,3662340,3270024,4054541,3961,3971,3675339,2137162,
                 4054540,3662339,2137159,3675338,3426774,4004,12260,3501804,3675337,4054539,
                 3929852,3342908,3426772,2157641,3662341,3229726,4150433,46222])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 6 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [25]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [26]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [27]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [28]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_79946_sms_viber_tg,94118,"118,546.31",1.36,"127,852.44",1.47,"137,940.00","3,378,798.95","69,966.00","21,408.00",86560,...,"351,281.00",0.01,1.00,0.69,"65,036.00",0.10,1.00,0.68,"64,219.00",0.63
0_79909_sms_viber_cg,94437,"21,883.81",0.00,0.00,0.00,0.00,"3,355,227.60","69,027.00","21,086.00",86619,...,"-381,670.00",-0.01,0.00,0.18,"17,293.00",0.03,0.00,0.29,"27,159.00",0.45
1_79909_sms_viber_cg,40603,"20,928.89",0.00,0.00,0.00,0.00,"1,455,599.63","29,475.00","8,427.00",37304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')